In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [212]:
# SSH tunnel configuration
SSH_HOST='ec2-65-1-131-4.ap-south-1.compute.amazonaws.com'
SSH_USER='ec2-user'
SSH_KEY_PATH='/Users/ravi.tej/Downloads/bastion-host-key-pair-bkp.pem'
LOCAL_BIND_PORT=3001

# MongoDB server configuration
MONGO_HOST=''
MONGO_PORT=27017
MONGO_USERNAME=''
MONGO_PASSWORD=''


MONGO_DB_NAME='insight_db'
MONGO_COLLECTION_NAME='llm_responses'

# test environment
ENVIRONMENT='local'

In [213]:
DB_USERNAME = MONGO_USERNAME
DB_PASSWORD=MONGO_PASSWORD

In [214]:
import os

In [215]:

import os
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder

# # SSH tunnel configuration
# SSH_HOST = os.environ.get('SSH_HOST')
# SSH_USER = os.environ.get('SSH_USER')
# SSH_KEY_PATH = os.environ.get('SSH_KEY_PATH')
# LOCAL_BIND_PORT = int(os.environ.get('LOCAL_BIND_PORT', 3010))


# # MongoDB server configuration
# MONGO_HOST = os.environ.get('MONGO_HOST')
# MONGO_PORT = int(os.environ.get('MONGO_PORT', 27017))
# DB_USERNAME = os.environ.get('MONGO_USERNAME')
# DB_PASSWORD = os.environ.get('MONGO_PASSWORD')

# db parameters dict
DB_PARAMS = {
    "host": '127.0.0.1',
    "port": LOCAL_BIND_PORT,
    "username": DB_USERNAME,
    "password": DB_PASSWORD,
}


class MongoDatabaseConnection:
    _instance = None
    _client = None
    _tunnel = None

    def __new__(cls):
        if cls._instance is None:
            try:
                cls._instance = super(MongoDatabaseConnection, cls).__new__(cls)

                if os.environ.get('ENVIRONMENT') == 'local':
                    # initiate mongo client via ssh tunneling
                    cls._instance._client = cls._connect_to_mongodb_using_ssh()
                else:
                    # initiate mongo client
                    cls._instance._client = cls._connect_to_mongodb()
            except Exception as e:
                raise Exception(f"Failed to connect to MongoDB: {e}")

        return cls._instance

    @classmethod
    def get_client(cls):
        return cls._instance._client

    @classmethod
    def close_connection(cls):
        if cls._instance._client:
            cls._instance._client.close()
            cls._instance._client = None

        if cls._tunnel:
            cls._tunnel.stop()
            cls._tunnel = None

    @classmethod
    def _establish_tunnel(cls):
        try:
            tunnel = SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username=SSH_USER,
                ssh_pkey=SSH_KEY_PATH,
                remote_bind_address=(MONGO_HOST, MONGO_PORT),
                local_bind_address=('127.0.0.1', LOCAL_BIND_PORT)
            )

            return tunnel
        except Exception as e:
            raise Exception(f"Failed to establish SSH tunnel: {e}")

    @classmethod
    def _connect_to_mongodb(cls):
        try:
            return MongoClient(**DB_PARAMS)
        except ConnectionError as mongo_error:
            raise Exception(f"Failed to connect to MongoDB Connection: {mongo_error}")

    @classmethod
    def _connect_to_mongodb_using_ssh(cls):
        try:
            # start the tunnel
            cls._tunnel = cls._establish_tunnel()
            cls._tunnel.start()

            return MongoClient(
                directConnection=True,
                **DB_PARAMS
            )
        except ConnectionError as mongo_error:
            raise Exception(f"Failed to connect to MongoDB Connection: {mongo_error}")

In [216]:
from bson import ObjectId

In [231]:
class MongoDBArticle:

    @staticmethod
    def get_db():
        mongo_conn = None
        try:
            # setup mongo connection
            mongo_conn = MongoDatabaseConnection()
            mongo_client = mongo_conn.get_client()
            db = mongo_client[MONGO_DB_NAME]
            return db
        except Exception as e:
            if mongo_conn:
                mongo_conn.close_connection()
            raise Exception(f"Error in connecting to db: {e}")

    @staticmethod
    def get_collection(collection_name=None):
        try:
            db = MongoDBArticle.get_db()
            if collection_name is None:
                return db[MONGO_COLLECTION_NAME]
            else:
                return db[collection_name]
        except Exception as e:
            raise Exception(f"Error in _get_collection: {e}")

    @staticmethod
    def fetch_documents_by_ids(string_ids):
        try:
            object_ids = [ObjectId(string_id) for string_id in string_ids]
        except Exception as e:
            return f"Invalid ID format: {e}"

        collection = MongoDBArticle.get_collection()
        documents = list(collection.find({"_id": {"$in": object_ids}}))
        doc_dict = {}
        for doc in documents:
            article_id = str(doc.pop('_id'))
            doc_dict[article_id] = doc
            doc_dict[article_id]['article_id'] = article_id
            doc_dict[article_id]['source_id'] = str(doc['source_id'])
            doc_dict[article_id]['is_premium_article'] = False
        return doc_dict
    
    @staticmethod
    def fetch_all_documents():
        query = {"is_premium_article": False}
        # projection = {"_id": 1}; , projection

        collection = MongoDBArticle.get_collection()
        documents = list(collection.find(query))
        doc_dict = {}
        for doc in documents:
            article_id = str(doc.pop('_id'))
            doc_dict[article_id] = doc
            doc_dict[article_id]['article_id'] = article_id
        return doc_dict
    
    @staticmethod
    def fetch_all_llm_responses():
        query = {"response": {"$exists": True}}
        # projection = {"_id": 1}; , projection

        collection = MongoDBArticle.get_collection()
        documents = list(collection.find(query))
        doc_dict = {}
        for doc in documents:
            _id = str(doc.pop('_id'))
            doc_dict[article_id] = doc['article_id']
            doc_dict[article_id] = doc
            doc_dict[article_id]['_id'] = _id
            try:
                binary_data = base64.b64decode(doc['response'])
                parsed_response = json.loads(json.loads(binary_data)[0]['generated_text'])
            except:
                parsed_response = {}
                parsed_response['top_categories'] = ''
            doc_dict['parsed_response'] = parsed_response
        return doc_dict

### Extracting tags

In [129]:
docs = MongoDBArticle.fetch_all_documents()

In [125]:
list(docs.keys())[0]

'651dd3a8a662d76276b7db6a'

In [145]:
docs['651dd3a8a662d76276b7db6a']['tags']

['michael mosley',
 'india',
 'rajeev chandrasekar',
 'digital india',
 'government backing',
 'industry development',
 'cutting-edge technology',
 'ministry of electronics and information technology',
 'indian isa',
 'iit madras research park']

In [147]:
from tqdm import tqdm

In [184]:
tags_dict = {}

for doc in tqdm(list(docs.values())):
    cur_tags = doc.get('tags',[])
    if not cur_tags:
        continue
    for cur_tag in cur_tags:
        cur_tag = str(cur_tag).lower()
        all_cur_tags = cur_tag.split(',')
        for tag in all_cur_tags:
            tag = tag.strip()
            tags_dict[tag] = tags_dict.get(tag,0) + 1

100%|██████████| 230113/230113 [00:00<00:00, 589080.29it/s]


In [185]:
len(tags_dict)

267613

In [186]:
# Sorting the dictionary by values in descending order
sorted_tags = dict(sorted(tags_dict.items(), key=lambda item: item[1], reverse=True))

In [187]:
list(sorted_tags.keys())[:1000]

['',
 'results',
 'earnings first-cut',
 'news',
 'business news',
 'business',
 'personal finance',
 'entertainment',
 'markets',
 'economy',
 'india',
 'travel',
 'health',
 'investing',
 'stock market today',
 'markets news',
 'politics',
 'stock market',
 'technology',
 'recommendations',
 'china',
 'real estate',
 'sensex',
 'nifty',
 'united states',
 'breaking news: politics',
 'reviews',
 'company news',
 'inflation',
 'stock market news',
 'breaking news: business',
 'trading',
 'breaking news: markets',
 'rbi',
 'zacks rank',
 'breaking news: technology',
 'cmpny',
 'buy',
 'congress',
 'us',
 'investment strategy',
 'publ',
 'insurance',
 'stock markets',
 'food',
 'earnings',
 'joe biden',
 'buzzing stocks',
 'fin',
 'parenting',
 'bjp',
 'narendra modi',
 'bse',
 'amers',
 'interest rates',
 'nifty50',
 'namer',
 'nifty 50',
 'government and politics',
 'stocks',
 'donald trump',
 'companies',
 'culture',
 'gen',
 'asia',
 'sebi',
 'europ',
 's&p 500',
 'nse',
 'topnws',
 

In [190]:
import base64

In [192]:
# a = {
#   "_id": {
#     "$oid": "656dc4d9ff8e8d9bd0983b41"
#   },
#   "request_id": "393c3916-7734-40b5-ac7b-69123367882f",
#   "article_id": "6555c7e34b13023f9348ea91",
#   "params": {
#     "do_sample": true,
#     "top_p": 0.9,
#     "temperature": 1,
#     "max_new_tokens": 1024,
#     "repetition_penalty": 1.03,
#     "stop": [
#       "###",
#       "</s>",
#       "<|im_end|>"
#     ],
#     "article_id": "6555c7e34b13023f9348ea91"
#   },
#   "endpoint": "OpenHermes-Finetune-WatermelonSapphireZipline",
#   "model_finetune_id": "OpenHermes_WatermelonSapphireZipline",
#   "prompt": "<|im_start|>system\nYou are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.\n1. analysis_of_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>\n2. financial_or_business_news (True or False) : <True or False based on previous attribute>\n3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example international articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or important news of large multinational corporations will be relevant>\n4. relevant_for_india (True or False) : <True or False based on previous attribute>\n5. analysis_of_article_validity_duration (short text) : <analyse relevance duration. Be stingy: Stock fluctuations, 1 day; significant policy changes - few days or a week; educational content with references to any regulations is 30 unless there are none - in which case is timeless. International news in India has shorter lifespan. breaking news are usually not timeless; quarterly analysis or results are usually valid for a 3 days, yearly analysis or results for a weeks and a much longer one for a month.>\n6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>\n7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>\n8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>\n9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles pass on information on events. opinion pieces have inferences or predictions either from the author or from statements without data. analysis pieces have substantial data to justify their inferences or predictions. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>\n10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>\n11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>\n12. article_sentiment (one of bull, bear, na): <based on previous attribute>\n13. headline_suggestion (short text) : <Write a headline based on the content of the article>\n14. first_attempt_summary (text of 60 words) : <Generate concise, entity-dense summary. The summary should become highly dense but easily understood without the Article. Don't keep the summary too short, but limit it to no more than 60 words>\n15. improved_summary (text of 60 words): <Identify contents of the article which are missing from the previous summary but are important part of the article>\n16. final_summary (text of 60 words): <The finalised summary which is a mixture of first_attempt_summary and improved_summary. This summary should be very dense and also all the important information of the article and yet concise. By reading this in most cases the users need not read the article>\n17. top_categories (5 semi colon seperated categories): <Hierarchy of 5 categories or keywords. Start with a generic category and make it progressively specific. Select only 5 and seperate them by (;). Don't use either single or double quotes at any cost to avoid json.loads() failure>\nyour response should be a json structure with all the 17 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. respond in the exact following structure:\n{\n\"analysis_of_financial_or_business_news\": \"\",\n\"financial_or_business_news\": \"\",\n\"analysis_of_relevant_for_india\": \"\",\n\"relevant_for_india\": \"\",\n\"analysis_of_article_validity_duration\": \"\",\n\"article_validity_duration\": \"\",\n\"analysis_of_popularity\": \"\",\n\"popularity\": \"\",\n\"analysis_of_article_type\": \"\",\n\"article_type\": \"\",\n\"analysis_of_article_sentiment\": \"\",\n\"article_sentiment\": \"\",\n\"headline_suggestion\": \"\",\n\"first_attempt_summary\": \"\",\n\"improved_summary\": \"\",\n\"final_summary\": \"\",\n\"top_categories\": \"\"\n}\n<|im_end|>\n<|im_start|>user\n|article_start|\n How Buyers Are Financing Holiday Purchases This Year: Despite high inflation trends and concerns about a potential recession shoppers spent at record levels online on Cyber Monday and Black Friday amid a flurry of steep discounts thanks in part to buy now pay later (BNPL) apps. BNPL apps partner with retailers to make it easier for consumers to buy products online then pay for them over time. BNPL orders were up 78% from Nov. 19 to Nov. 25 compared to the week prior according to Adobe Analytics. The firm tracks more than 85% of the top 100 internet retailers in the U.S. About 40% of consumers said they will finance one or more of their holiday purchases this year including with credit cards personal loans and BNPL apps according to a study by LendingTree and PYMTS. And the likelihood that consumers will finance their purchases increases with their financial strain. About 52% of people who reported they live paycheck-to-paycheck or who struggle to pay their bills plan to finance holiday spending. Thanksgiving Holidays Record-Breaking Sales Consumers making purchases online spent a record $11.3 billion on Cyber Monday up from 5.8% from spending on the same day last year according to Adobe Analytics which tracks more than 85% of the top 100 internet retailers in the U.S. Spending on Black Friday hit $9.12 billion up 2.3% from Black Friday last year. Meanwhile shoppers also flocked back to stores in record numbers over the holiday weekend. From Thanksgiving Day to Cyber Monday 19.7 million shoppers returned to in-person shopping according to the National Retail Federation. Overall with both online and in-person shopping a record 196.7 shoppers made purchases over that five-day period up 9% from the same period last year. The NRF had expected 166.3 million shoppers. As inflationary pressures persist consumers have responded by stretching their dollars in any way possible Matthew Shay NRF president and CEO said in a statement. Retailers have responded accordingly offering shoppers a season of buying convenience matching sales and promotions across online and in-store channels.\n|article_end|\n<|im_end|>\n<|im_start|>assistant\n",
#   "response": {
#     "$binary": {
#       "base64": ,
#       "subType": "00"
#     }
#   }
# }

In [193]:
a = "W3siZ2VuZXJhdGVkX3RleHQiOiJ7XCJhbmFseXNpc19vZl9hcnRpY2xlX3NlbnRpbWVudFwiOiBcIlRoZSBhcnRpY2xlIHJlcG9ydHMgYSBwb3NpdGl2ZSB0cmVuZCBpbiBvbmxpbmUgc2hvcHBpbmcsIGluZGljYXRpbmcgYSBwb3NpdGl2ZSBtYXJrZXQgb3V0bG9vay5cIiwgXCJhbmFseXNpc19vZl9hcnRpY2xlX3R5cGVcIjogXCJUaGUgYXJ0aWNsZSBpcyBmYWN0LWJhc2VkLCByZXBvcnRpbmcgb24gY29uc3VtZXIgYmVoYXZpb3IgYW5kIHJldGFpbCB0cmVuZHMgZHVyaW5nIGhvbGlkYXkgc2hvcHBpbmcuXCIsIFwiYW5hbHlzaXNfb2ZfYXJ0aWNsZV92YWxpZGl0eV9kdXJhdGlvblwiOiBcIlRoZSBhcnRpY2xlIGRpc2N1c3NlcyBhIHNwZWNpZmljIGV2ZW50IChDeWJlciBNb25kYXkgYW5kIEJsYWNrIEZyaWRheSkgYW5kIHRodXMgaG9sZHMgcmVsZXZhbmNlIGZvciBhIGZldyBkYXlzLlwiLCBcImFuYWx5c2lzX29mX2ZpbmFuY2lhbF9vcl9idXNpbmVzc19uZXdzXCI6IFwiVGhlIGFydGljbGUgZGlzY3Vzc2VzIHRoZSByaXNlIGluIHVzZSBvZiAnQnV5IE5vdyBQYXkgTGF0ZXInIHNjaGVtZXMgaW4gaG9saWRheSBzaG9wcGluZ1wiLCBcImFuYWx5c2lzX29mX3BvcHVsYXJpdHlcIjogXCJUaGlzIGFydGljbGUgY292ZXJzIGhvbGlkYXkgc2hvcHBpbmcgYW5kIHBvcHVsYXIgJ0J1eSBOb3cgUGF5IExhdGVyJyBzY2hlbWVzLCBhIHBvcHVsYXIgdG9waWMgYW1vbmcgSW5kaWFuIGNvbnN1bWVyc1wiLCBcImFuYWx5c2lzX29mX3JlbGV2YW50X2Zvcl9pbmRpYVwiOiBcIkRlc3BpdGUgYmVpbmcgZm9jdXNlZCBvbiBVUywgdGhlIHJpc2Ugb2YgJ0J1eSBOb3cgUGF5IExhdGVyJyBzY2hlbWVzIGNhbiBiZSByZWxldmFudCBmb3IgSW5kaWFuIGNvbnN1bWVycyBhcyB0aGV5IGZpbmQgdGhlbXNlbHZlcyBmaW5hbmNpYWxseSBzdHJhaW5lZCBhbWlkIGluZmxhdGlvbi5cIiwgXCJhcnRpY2xlX3NlbnRpbWVudFwiOiBcImJ1bGxcIiwgXCJhcnRpY2xlX3R5cGVcIjogXCJmYWN0XCIsIFwiYXJ0aWNsZV92YWxpZGl0eV9kdXJhdGlvblwiOiA3LCBcImZpbmFsX3N1bW1hcnlcIjogXCJJbiBhIHJlbWFya2FibGUgaG9saWRheSBzaG9wcGluZyBzZWFzb24sIHRoZSB1c2Ugb2YgJ0J1eSBOb3cgUGF5IExhdGVyJyAoQk5QTCkgc2NoZW1lcyByb3NlLCBkcml2ZW4gYnkgcmVjb3JkLWJyZWFraW5nIEN5YmVyIE1vbmRheSBhbmQgQmxhY2sgRnJpZGF5IHNhbGVzLiBUaGlzIHRyZW5kIHdhcyBtb3N0IHNpZ25pZmljYW50IGFtb25nIHRob3NlIGZpbmFuY2lhbGx5IHN0cmFwcGVkLCB3aXRoIDUyJSBwbGFubmluZyB0byB1c2Ugc3VjaCBtZXRob2RzIHRvIGZ1bmQgdGhlaXIgaG9saWRheSBzaG9wcGluZy4gU2hvcHBlcnMgZW1icmFjZWQgaW4tcGVyc29uIHNob3BwaW5nIGFzIHdlbGwsIHdpdGggMTkuNyBtaWxsaW9uIGN1c3RvbWVycyB2aXNpdGluZyBzdG9yZXMgZHVyaW5nIFRoYW5rc2dpdmluZyB3ZWVrZW5kLlwiLCBcImZpbmFuY2lhbF9vcl9idXNpbmVzc19uZXdzXCI6IHRydWUsIFwiZmlyc3RfYXR0ZW1wdF9zdW1tYXJ5XCI6IFwiSG9saWRheSBzaG9wcGVycyB3ZXJlIGFibGUgdG8gYnV5IG9ubGluZSBvbiBCbGFjayBGcmlkYXkgYW5kIEN5YmVyIE1vbmRheSB0aGFua3MgdG8gdGhlIHJpc2Ugb2YgJ0J1eSBOb3cgUGF5IExhdGVyJyAoQk5QTCkgc2NoZW1lcy4gQWNjb3JkaW5nIHRvIEFkb2JlIEFuYWx5dGljcywgQk5QTCBvcmRlcnMgcm9zZSA3OCUgYmV0d2VlbiBOb3YuIDE5LTI1IGZyb20gdGhlIHByZXZpb3VzIHdlZWsuIExlbmRpbmdUcmVlIGFuZCBQWU1UUyByZXBvcnRlZCB0aGF0IDQwJSBvZiBjb25zdW1lcnMgd291bGQgYmUgdXNpbmcgY3JlZGl0IGNhcmRzLCBwZXJzb25hbCBsb2Fucywgb3IgQk5QTCBzY2hlbWVzIHRvIGZ1bmQgdGhlaXIgaG9saWRheSBzaG9wcGluZywgZXNwZWNpYWxseSB0aG9zZSBmaW5hbmNpYWxseSBzdHJhaW5lZC4gU2hvcHBpbmcgYWxzbyBwaWNrZWQgdXAgaW4gc3RvcmVzLCB3aXRoIDE5LjcgbWlsbGlvbiBzaG9wcGVycyB2aXNpdGluZyBwaHlzaWNhbCBzdG9yZXMuXCIsIFwiaGVhZGxpbmVfc3VnZ2VzdGlvblwiOiBcIkhvbGlkYXkgU2hvcHBpbmcgQm9vbSBXaXRoIFJpc2UgaW4gQnV5IE5vdyBQYXkgTGF0ZXIgU2NoZW1lc1wiLCBcImltcHJvdmVkX3N1bW1hcnlcIjogXCJUaGUgYm9vbSBpbiBvbmxpbmUgc2hvcHBpbmcgZHVlIHRvIGhvbGlkYXkgc2FsZXMgd2FzIGFzc2lzdGVkIGJ5IEJOUEwgc2NoZW1lcy4gT2YgdGhlIDUyJSBvZiBjb25zdW1lcnMgcGxhbm5pbmcgdG8gZnVuZCB0aGVpciBob2xpZGF5IHNob3BwaW5nIHRocm91Z2ggQk5QTCBzY2hlbWVzLCA0MCUgc2FpZCB0aGV5IGRpZCBzbyBiZWNhdXNlIG9mIHRoZWlyIGZpbmFuY2lhbCBzdHJlc3MuIFNob3BwZXJzIGFsc28gcmV0dXJuZWQgdG8gaW4tcGVyc29uIHNob3BwaW5nIG92ZXIgdGhlIGhvbGlkYXkgd2Vla2VuZCB3aXRoIDE5LjcgbWlsbGlvbiBzaG9wcGVycyB2aXNpdGluZyBzdG9yZXMuXCIsIFwicG9wdWxhcml0eVwiOiBcIm1vZGVyYXRlbHlfcG9wdWxhclwiLCBcInJlbGV2YW50X2Zvcl9pbmRpYVwiOiB0cnVlLCBcInRvcF9jYXRlZ29yaWVzXCI6IFwiRmluYW5jZTtSZXRhaWw7T25saW5lIFNob3BwaW5nO0J1eSBOb3cgUGF5IExhdGVyO0NvbnN1bWVyIEJlaGF2aW9yXCJ9In1d"

In [194]:
binary_data = base64.b64decode(a)

In [199]:
json.loads(json.loads(binary_data)[0]['generated_text'])

{'analysis_of_article_sentiment': 'The article reports a positive trend in online shopping, indicating a positive market outlook.',
 'analysis_of_article_type': 'The article is fact-based, reporting on consumer behavior and retail trends during holiday shopping.',
 'analysis_of_article_validity_duration': 'The article discusses a specific event (Cyber Monday and Black Friday) and thus holds relevance for a few days.',
 'analysis_of_financial_or_business_news': "The article discusses the rise in use of 'Buy Now Pay Later' schemes in holiday shopping",
 'analysis_of_popularity': "This article covers holiday shopping and popular 'Buy Now Pay Later' schemes, a popular topic among Indian consumers",
 'analysis_of_relevant_for_india': "Despite being focused on US, the rise of 'Buy Now Pay Later' schemes can be relevant for Indian consumers as they find themselves financially strained amid inflation.",
 'article_sentiment': 'bull',
 'article_type': 'fact',
 'article_validity_duration': 7,
 'f

In [189]:
sorted_tags['business']

10615

### Getting llm attributes

In [160]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [161]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT * FROM llm_article_attributes'
    cursor.execute(sql)
    results = cursor.fetchall()

In [177]:
cols = '''"id","article_id"	"financial_news"	"relevant_for_india"	"validity_duration"	"expected_popularity"	"article_type"	"sentiment"	"model_finetune_id"	"created_at"	"updated_at"	"interest_score"	"headline_score"	"novelty_score"	"emotional_score"	"india_flag"'''

cols = cols.replace('\t',',')

cols = [str(x).replace('"','') for x in cols.split(',')]

import pandas as pd

article_attributes = pd.DataFrame(results,columns=cols)

valid_fin_articles = article_attributes[(article_attributes['financial_news'] == True) & (article_attributes['relevant_for_india']==True)]

In [178]:
len(valid_fin_articles)

167765

In [232]:
llm_responses = MongoDBArticle.fetch_all_llm_responses()

In [16]:
text[2]

'title: XIRR vs CAGR: Which is the right way to calculate mutual fund returns? | Mint. published_date: 2023-10-19T11:42:31+05:30. content:  Before you start investing in the mutual funds, you should ideally examine the past returns of all the schemes that you want to invest into. And since there are a number of ways to calculate the past returns, you should opt for the one that suits you the most. For instance, XIRR and CAGR are among the two most preferred parameters that are used to calculate the returns on various investment products including mutual fund returns. Most investors get confused as to which parameter should one go for. Why should one measure the return on investment on their mutual fund investment in the first place? To find this out we have to\xa0explore XIRR and CAGR in detail to understand which method is the right way to calculate mutual fund returns. What is XIRR? XIRR\xa0stands for extended internal rate of return. It is a method that is used to calculate the annu

In [124]:
len(docs)

230113

In [8]:
article_ids = MongoDBArticle.fetch_all_document_ids()

In [10]:
article_docs = MongoDBArticle.fetch_documents_by_ids(article_ids)

In [11]:
from src.Article import Article

In [12]:
articles = {}
for id, article in article_docs.items():
    articles[id] = Article.from_dict(article)

### Calling server for results

In [6]:
import requests
import json

In [7]:
r = requests.post('http://hubble-env.eba-pguqm8s4.ap-south-1.elasticbeanstalk.com/search_query', json={'query_text': 'what is xirr'})

In [5]:
from datetime import datetime

In [6]:
def get_max_published_date(max_date):
    if not max_date:
        # Get the current date and time
        now = datetime.now()
        # Replace the time part with 23:59:59
        end_of_day = now.replace(hour=23, minute=59, second=59, microsecond=0)
        # Convert to a string (e.g., 'YYYY-MM-DD 23:59:59')
        date_string = end_of_day.strftime('%Y-%m-%d %H:%M:%S')
        return date_string
    else:
        return max_date + ' 23:59:59'

In [7]:
get_max_published_date('2024-01-06')

'2024-01-06 23:59:59'

In [8]:
get_max_published_date(None)

'2024-01-25 23:59:59'

In [39]:
'2024-01-08T15:31:25+05:30' > get_max_published_date('2024-01-12')

False

In [11]:
best_results = [x['article_id'] for x in json.loads(r.text)['searchArticleIds'][:5]]

### Getting articles from s3

In [2]:
from src.ArticleService import ArticleService

In [24]:
text = []
for article_id in best_results:
    cur_text = ArticleService.get_article_content_from_s3(article_id)
    cur_words = cur_text.split(' ')[:384]
    cur_text = ' '.join(cur_words)
    text.append(cur_text)

In [95]:
ArticleService.get_article_content_from_s3('6555b4c34b13023f9348b7a5')

'title: What is XIRR in Mutual Funds: Calculation & Formula in Excel. published_date: 2023-02-13T11:50:00+00:00. content: XIRR or Extended Internal Rate of Return helps calculate the return on mutual fund investments made through SIP (Systematic Investment Plan). CEO of Navi Mutual Fund, Hari Shyamsunder says, “Your investments should always be compared against a possible alternative. But to make that comparison, you must understand the basic concept of XIRR.” This article is a beginner’s guide to understanding the XIRR meaning in mutual funds, its importance and the calculation of your SIP returns. Read on! What is XIRR in Mutual Funds? XIRR Meaning – XIRR (Extended Internal Rate of Return) can be defined as a computation method for calculating returns of mutual fund investments at irregular intervals. It can also be defined as a single rate of return which can be applicable for every instalment and redemption. You can use it to have an understanding of the current value of all your i

In [16]:
text[2]

'title: XIRR vs CAGR: Which is the right way to calculate mutual fund returns? | Mint. published_date: 2023-10-19T11:42:31+05:30. content:  Before you start investing in the mutual funds, you should ideally examine the past returns of all the schemes that you want to invest into. And since there are a number of ways to calculate the past returns, you should opt for the one that suits you the most. For instance, XIRR and CAGR are among the two most preferred parameters that are used to calculate the returns on various investment products including mutual fund returns. Most investors get confused as to which parameter should one go for. Why should one measure the return on investment on their mutual fund investment in the first place? To find this out we have to\xa0explore XIRR and CAGR in detail to understand which method is the right way to calculate mutual fund returns. What is XIRR? XIRR\xa0stands for extended internal rate of return. It is a method that is used to calculate the annu

In [3]:
a = ArticleService.get_article_content_from_s3('659bd5c24b25ce4814fb5155')

In [21]:
from datetime import datetime

In [4]:
a

"title: Top Gainers and Losers today on 8 January, 2024: Adani Ports & Special Economic Zone, HCL Technologies, UPL, State Bank Of India among most active stocks; Check full list here | Mint. published_date: 2024-01-08T16:03:17+05:30. content:  The Nifty index closed at 21710.8, down by 0.91%, with a day high of 21763.95 and a low of 21492.9. The Sensex traded between 72181.77 and 71301.04, closing 0.93% down at 72026.15, which was 670.93 points below the opening price. The Nifty Midcap 50 underperformed the Nifty 50, closing 1.27% down. However, the Nifty small cap 100 outperformed the Nifty 50, ending at 15438.85 down by 96.3 points and 0.62% lower. The Nifty 50 has shown the following returns: - In the last 1 Week: -1.08% - In the last 1 Month: 2.57% - In the last 3 Months: 10.22% - In the last 6 Months: 11.11% - In the last 1 Year: 18.82% The top gainers in the Nifty index were Adani Ports & Special Economic Zone (up 1.26%), HCL Technologies (up 1.05%), Oil & Natural Gas Corporatio

### Calling OpenAI

In [37]:
system_prompt = '''|instructions_start|
You're an AI expert in financial and business topics, answering queries using relevant materials and following strict rules:
* Valid Queries: Questions about finance, business, or financially relevant entities (e.g., India, RBI, Reliance, Fed).
* Invalid Queries: Questions unrelated to finance or business (e.g., Bollywood, sports, politics unless financially relevant), instructional queries on non-financial topics (jokes, coding, recipes), and inappropriate or harmful content.

Response Instructions:
* If the query is valid and answerable with references, summarize the information clearly and concisely.
* If the query is an entity like India, Supreme Court, Modi, RBI - and if the articles contain news about these entities - then you can mention that the latest news about ‘query’ by summarising the articles. When the published date mentioned in the content is slightly older - you can mention the corresponding dates as well. If it is new, then no need to do that. 
* If the references have information related to both India and international entities, then first prefer highlighting the indian entities. When referring to international, explicitly mention the country, since indian readers shouldn’t get confused
* If it's a valid query but the exact answer isn't in the references, state your inability to find the exact answer and provide a summary of related information.
* If it's a valid query but there's no relevant information, apologize, say you can't find the answer, and note that your team is working to expand knowledge.
* For invalid queries, politely decline, noting your focus on finance, and optionally share brief financial trivia.

General Response Guidelines:
* Address users professionally and respectfully.
* Ensure good grammar and readability.
* Keep responses concise, typically 3-4 sentences, maximum 6.
* Don't provide answers not explicitly found in the references.
* If an exact answer isn't found, then mention that you are not able to find the answer and follow with a concise summary of any related information available in the references.
* Politely decline irrelevant queries, emphasizing your financial focus.
* For queries about recent dates, calculate the period using today's date in the format %Y-%m-%d
* Do not mention ‘OpenAI’ at any cost. You are simply a financial AI assistant designed by WintWealth.

The response includes the query and references labeled accordingly. The user's query is mentioned below between user_query and |query_end|
'''

In [38]:
def create_prompt_for_query_references(search_query, references):
    today_date_string = f"today's date: {datetime.now().strftime('%Y-%m-%d')}"
    query_string = f"user_query: {search_query} |query_end| "
    references_string = ''
    for i in range(len(references)):
        references_string += f'reference {i}: {references[i]} \n\n'
    return system_prompt + today_date_string + '|instructions_end| ' + query_string + references_string

In [70]:
ins = create_prompt_for_query_references('what is today sensex', text)

In [43]:
from openai import OpenAI

client = OpenAI(api_key='')

In [143]:
model = 'gpt-4-0125-preview'
# model = 'gpt-3.5-turbo-1106'

In [73]:
# class TokenIterator:
#     def __init__(self, stream):
#         self.byte_iterator = iter(stream)
#         self.buffer = io.BytesIO()
#         self.read_pos = 0

#     def __iter__(self):
#         return self

#     def __next__(self):
#         while True:
#             self.buffer.seek(self.read_pos)
#             line = self.buffer.readline()
#             if line and line[-1] == ord("\n"):
#                 self.read_pos += len(line) + 1
#                 full_line = line[:-1].decode("utf-8")
#                 line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
#                 return line_data["token"]["text"]
#             chunk = next(self.byte_iterator)
#             self.buffer.seek(0, io.SEEK_END)
#             self.buffer.write(chunk["PayloadPart"]["Bytes"])
import io
import json

class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line)
                # Decode and parse the JSON line, removing the expected prefix and suffix
                full_line = line[:-1].decode("utf-8")
                line_data = json.loads(full_line.lstrip("data:").rstrip("\n"))
                return line_data["token"]["text"]

            # Retrieve the next chunk and write its bytes to the buffer
            chunk = next(self.byte_iterator)

            # Extracting the specific bytes from the chunk
            bytes_to_write = chunk.choices[0].delta.content

            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(bytes_to_write)

In [141]:
res = client.chat.completions.create(model = model, messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': ins}], stream=False)
# for chunk in res:
#     current_content = chunk.choices[0].delta.content
#     if current_content:
#         print(current_content)
    # cur_str = chunk.choices[0].delta.content
    # if cur_str:
    #     print(cur_str)

In [142]:
res.choices[0].message.content

"I'm sorry, but I cannot provide the latest sensex value as the information is not available in the provided references. However, I can share that the calculation of XIRR (Extended Internal Rate of Return) helps in understanding the returns on mutual fund investments, especially when transactions occur at irregular intervals. It accounts for individual cash flows on different dates, providing a more accurate method of calculating returns. If you have any more financial queries, feel free to ask!"

In [136]:
res

ChatCompletion(id='chatcmpl-8l5Tbmfs9KUerX2svJmaanAK72T5w', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm unable to find the specific information on today's Sensex movements or values within the references provided. For real-time updates or today's numbers, I would recommend checking a reliable financial news website or a stock market tracking platform. My resources are limited to the information provided in the references, which do not include current stock market updates.", role='assistant', function_call=None, tool_calls=None))], created=1706233159, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_376b7f78b9', usage=CompletionUsage(completion_tokens=67, prompt_tokens=3059, total_tokens=3126))

In [139]:
3126 * 0.03/1000

0.09378

In [138]:
0.03 * 10000

300.0

In [54]:
res = client.chat.completions.create(model = model, messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': ins}])

In [55]:
res.choices[0].message.content

"The XIRR or Extended Internal Rate of Return is a method for calculating the return on mutual fund investments, especially those that involve irregular transactions, such as investments via a Systematic Investment Plan (SIP). It serves as a way to understand the actual rate of return on investments that have varying cash flow timings, allowing you to compare your existing investments with potential alternatives. XIRR considers the timing of each cash outflow (like purchases and SIP instalments) and cash inflow (like dividends and redemptions), providing a more precise measurement of the annualized return than methods suited for regular intervals, such as CAGR.\n\nTo calculate XIRR, financial transactions and their corresponding dates are input into a formula in Microsoft Excel: `=XIRR(values, dates, [guess])`. In this formula, 'values' refer to the cash flows (negative for outflows, positive for inflows), 'dates' correspond to when the cash flows take place, and 'guess' is an optional

In [106]:
r = requests.post('http://hubble-env.eba-pguqm8s4.ap-south-1.elasticbeanstalk.com/search_query', json={'query_text': 'top stocks to invest in india right now'})

In [107]:
json.loads(r.text)['searchArticleIds']

[{'article_id': '65a62a493c864051fa47ec19'},
 {'article_id': '65683383207bb033b910ee35'},
 {'article_id': '6565c8b039bfc8784efe5b49'},
 {'article_id': '657aa8833a7a250d32265356'},
 {'article_id': '6555cc8f4b13023f934aa448'},
 {'article_id': '656e02799994965ac4b4b4e6'},
 {'article_id': '6555cbcc4b13023f934a1901'},
 {'article_id': '65316f821e5cc42b1b1404ab'},
 {'article_id': '656e925a9994965ac4b4d683'},
 {'article_id': '65a4d8b7ea435f28caee4ed7'},
 {'article_id': '65316f791e5cc42b1b140252'},
 {'article_id': '6555cbfb4b13023f934a3ad5'},
 {'article_id': '65316fae1e5cc42b1b141184'},
 {'article_id': '6555cc1f4b13023f934a53fa'},
 {'article_id': '65316fd11e5cc42b1b141db1'},
 {'article_id': '6555cc634b13023f934a8628'},
 {'article_id': '65316efc1e5cc42b1b13eb65'},
 {'article_id': '657fb5d58f821120f67a0986'},
 {'article_id': '6531688a1e5cc42b1b13af29'},
 {'article_id': '65316f021e5cc42b1b13ec1e'},
 {'article_id': '65316f911e5cc42b1b1408a9'},
 {'article_id': '65a91ec26391ac19baa842e5'},
 {'article

In [96]:
r1 = requests.get('http://hubble-env.eba-pguqm8s4.ap-south-1.elasticbeanstalk.com/summarize_search', params={'query': 'what is xirr','ids': '6555b4c34b13023f9348b7a5,652fcb091e5cc42b1b13a13e,6530d40c1e5cc42b1b13a6cf'})

In [97]:
r1.text

"XIRR, which stands for Extended Internal Rate of Return, is a method for calculating the return on investment for transactions that do not occur at regular intervals, such as those in mutual funds through a Systematic Investment Plan (SIP). It allows for the consideration of each individual cash flow along with the specific dates they occur, which gives a more accurate annualized rate of return for investments with irregular cash flows. The XIRR function is available in Microsoft Excel and can accommodate cash flows (both inflows and outflows) of varying amounts and at different times. It is particularly useful when comparing returns of investments that have different patterns of cash flows, such as additional investments or withdrawals made at various points in time. The formula provided in Microsoft Excel for XIRR is “= XIRR (values, dates, [guess])”, where 'values' refer to an array of cash flows, 'dates' correspond to the series of dates when the cash flows occurred, and 'guess' i

In [89]:
'6555b4c34b13023f9348b7a5,652fcb091e5cc42b1b13a13e,6530d40c1e5cc42b1b13a6cf'.split(',')

['6555b4c34b13023f9348b7a5',
 '652fcb091e5cc42b1b13a13e',
 '6530d40c1e5cc42b1b13a6cf']

In [88]:
r1.text

''

In [86]:
r2 = requests.get('http://streaming-sample-app-env.eba-5panndpt.ap-south-1.elasticbeanstalk.com/completion')

In [87]:
r2.text

'   The concept of reversing entropy is a part of the field of thermodynamics and is connected to the second law of thermodynamics. The second law of thermodynamis founds on the concept that the entropy or the total energy of a system tends to increase over time. It is also linked with the concept of the flow of heat from a hot body to a cooler one. When heat flows from a hot body to a cooler one it is termed as heat transfer and it is a process that is naturally initiated and follows the path of the natural gradient of temperature. Heating and cooling are natural processes that happen naturally and entropy tends to increase naturally. However there are certain instances where entropy can be reversed. These are the cases when the natural flow of heat is interrupted or reversed through a suitable device. These devices are called Heat pumps. These are devices that can reverse the direction of the flow of heat. They can remove heat from a low-temperature body and transfer it to a higher-t